In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.utils import shuffle

inp = pd.read_csv('Input/Cleaned_besk_eform.csv', dtype=object, encoding='iso-8859-1', names=['besk'], skiprows=[0]) 

nace=inp["besk"].str.split(";", n=1, expand=True)

nace=nace[nace[0].notnull()]
nace=shuffle(nace)


from sklearn.model_selection import train_test_split

besk=nace[0].values
sn07=nace[1].values

from sklearn import preprocessing
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
y_encode = lab_enc.fit_transform(sn07)

import keras

y = keras.utils.to_categorical(y_encode, 841)


from keras.preprocessing.text import Tokenizer

t = Tokenizer()
t.fit_on_texts(besk)
X=t.texts_to_sequences(besk)

vocab_size=len(t.word_index)+1

from keras.preprocessing.sequence import pad_sequences

maxlen=162

X_pad=pad_sequences(X, padding='post', maxlen=maxlen)


X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.5, random_state=1000)

#normalizer = preprocessing.Normalizer().fit(X_train)
#X_train_norm=normalizer.transform(X_train)
#X_test_norm=normalizer.transform(X_test)

#x_train= tf.keras.utils.normalize(X_train, axis=1)
#x_test= tf.keras.utils.normalize(X_test, axis=1)


#longest_string_sn07=max(sn07, key=len)
#len(longest_string_sn07)
#longest_string_besk=max(besk, key=len)
#len(longest_string_besk)
#max(y_encode)



In [ ]:
from keras.models import Sequential
from keras import layers
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, SeparableConv1D, MaxPooling1D
from keras.optimizers import SGD, Adam
from tensorflow.keras.utils import normalize
import tensorflow as tf
import keras
import time
from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import LearningRateScheduler
import math

# learning rate schedule
#def step_decay(epoch):
#    initial_lrate = 0.001
#    drop = 0.1
#    epochs_drop = 2
#    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
#    return lrate

#input_dim = X_train.shape[1:]
embedding_dim=50

dense_layers=[0,1,2]
conv_layers=[1,2,3]
layer_sizes=[64,128,256]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME="{}-dense-{}-conv-{}-layer-{}".format(dense_layer, conv_layer, layer_size, int(time.time()))
            tensorboard=TensorBoard(log_dir='logs/6/{}'.format(NAME))
            print(NAME)
            model=Sequential()
            model.add(layers.Embedding(input_dim=vocab_size,
                                       output_dim=embedding_dim,
                                       input_length=maxlen))
            model.add(layers.SeparableConv1D(layer_size ,5, activation='relu'))
            model.add(layers.MaxPooling1D(pool_size=2))

            for l in range(conv_layer-1):
                model.add(layers.SeparableConv1D(layer_size ,5, activation='relu'))
                model.add(layers.MaxPooling1D(pool_size=2))

            model.add(layers.Flatten())
            for l in range(dense_layer):
                model.add(layers.Dense(layer_size, activation='relu'))
                model.add(layers.Dropout(0.2))

            model.add(layers.Dense(841, activation='softmax'))
            epochs = 10
            learning_rate = 0.001
            decay_rate = learning_rate / epochs
            adam = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=decay_rate, amsgrad=False)
            model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
            #lrate = LearningRateScheduler(step_decay)
            callbacks_list = [tensorboard]
            model.fit(X_train, y_train,
                    epochs=10,
                    validation_data=(X_test, y_test), callbacks=callbacks_list)

#åpne tensorboard: tensorboard --logdi=ml/logs/

1-dense-1-conv-64-layer-1558011182
Train on 120709 samples, validate on 120709 samples
Epoch 1/10
120709/120709 [==============================] - 869s 7ms/step - loss: 4.3734 - acc: 0.1483 - val_loss: 3.6565 - val_acc: 0.2636
Epoch 2/10
120709/120709 [==============================] - 414s 3ms/step - loss: 3.4048 - acc: 0.2902 - val_loss: 3.2114 - val_acc: 0.3462
Epoch 3/10
120709/120709 [==============================] - 428s 4ms/step - loss: 2.9708 - acc: 0.3593 - val_loss: 3.0367 - val_acc: 0.3886
Epoch 4/10
120709/120709 [==============================] - 435s 4ms/step - loss: 2.6953 - acc: 0.4046 - val_loss: 2.9679 - val_acc: 0.4134
Epoch 5/10
120709/120709 [==============================] - 419s 3ms/step - loss: 2.4927 - acc: 0.4407 - val_loss: 2.9460 - val_acc: 0.4283
Epoch 6/10
120709/120709 [==============================] - 527s 4ms/step - loss: 2.3344 - acc: 0.4695 - val_loss: 2.9634 - val_acc: 0.4371
Epoch 7/10
120709/120709 [==============================] - 404s 3ms/step